In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

from tensorflow import keras
import tensorflowjs as tfjs
import pickle
import colorama

2022-10-19 14:20:35.678250: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-19 14:20:35.678302: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
with open('intents.json') as file:
  data = json.load(file)

In [3]:
data

{'intents': [{'tag': 0,
   'patterns': ['Pẹlẹ o ',
    ' Bawo ni o se wa?',
    ' Pẹlẹ o bawo ni ',
    ' Bawo ni nibe yen o ',
    ' kilode '],
   'responses': ['Pẹlẹ o',
    'Hello',
    ' Bawo ni o ṣe n ṣe',
    ' ikini fun ọ ',
    ' mo wa dada',
    'mo wa dada',
    'o dara o ṣeun']},
  {'tag': 1,
   'patterns': [' o dabọ ', ' ri ọ ', ' titi a o tun pade ', ' nigbamii '],
   'responses': ['O dara lati ba ọ sọrọ ', ' Ma a ri e laipe ', ' Sọ laipẹ!']},
  {'tag': 2,
   'patterns': ['si ilekun yẹn',
    ' pa ilẹkun yẹn',
    'si ilekun',
    'pa ilekun yen de',
    'silẹkun'],
   'responses': ['mo ti silekun ',
    'ilekun ti wa ni pipade',
    'mo ti pa ilekun yen de']},
  {'tag': 3,
   'patterns': ['tan fan yen', 'tan fan', 'Pa fan yen', 'Pa fan'],
   'responses': ['fan ti tan',
    'mo ti pa fan naa',
    'fan ti wa ni ti tan',
    'mo ti pa fan']},
  {'tag': 4,
   'patterns': ['tan ina yen', 'tan ina', 'Pa ina yen', 'Pa ina'],
   'responses': ['ina ti tan',
    'mo ti pa ina naa'

In [4]:
intents_mapping = {
  0: 'greeting',
  1: 'goodbye',
  2: 'command.door',
  3: 'command.fan',
  4: 'command.ina'
}

In [5]:
train_sentences = []
train_labels = []
labels = []
responses = []

In [6]:
for intent in data['intents']:
  for pattern in intent["patterns"]:
    train_sentences.append(pattern)
    train_labels.append(intent["tag"])
  responses.append(intent["responses"])

  if (intent["tag"] not in labels):
    labels.append(intent['tag'])

In [7]:
train_labels, train_sentences, labels, responses

([0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
 ['Pẹlẹ o ',
  ' Bawo ni o se wa?',
  ' Pẹlẹ o bawo ni ',
  ' Bawo ni nibe yen o ',
  ' kilode ',
  ' o dabọ ',
  ' ri ọ ',
  ' titi a o tun pade ',
  ' nigbamii ',
  'si ilekun yẹn',
  ' pa ilẹkun yẹn',
  'si ilekun',
  'pa ilekun yen de',
  'silẹkun',
  'tan fan yen',
  'tan fan',
  'Pa fan yen',
  'Pa fan',
  'tan ina yen',
  'tan ina',
  'Pa ina yen',
  'Pa ina'],
 [0, 1, 2, 3, 4],
 [['Pẹlẹ o',
   'Hello',
   ' Bawo ni o ṣe n ṣe',
   ' ikini fun ọ ',
   ' mo wa dada',
   'mo wa dada',
   'o dara o ṣeun'],
  ['O dara lati ba ọ sọrọ ', ' Ma a ri e laipe ', ' Sọ laipẹ!'],
  ['mo ti silekun ', 'ilekun ti wa ni pipade', 'mo ti pa ilekun yen de'],
  ['fan ti tan', 'mo ti pa fan naa', 'fan ti wa ni ti tan', 'mo ti pa fan'],
  ['ina ti tan', 'mo ti pa ina naa', 'ina ti wa ni ti tan', 'mo ti pa ina']])

In [8]:
num_classes = len(labels)
num_classes

5

In [9]:
# encoder = LabelEncoder()
# train_labels = encoder.fit_transform(train_labels)

In [10]:
train_labels

[0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4]

In [11]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

In [12]:
tokenizer = Tokenizer(
  num_words=vocab_size, oov_token=oov_token
)

In [13]:
tokenizer.fit_on_texts(train_sentences)

In [14]:
word_index = tokenizer.word_index
word_index

{'<OOV>': 1,
 'o': 2,
 'yen': 3,
 'pa': 4,
 'tan': 5,
 'fan': 6,
 'ina': 7,
 'bawo': 8,
 'ni': 9,
 'ilekun': 10,
 'pẹlẹ': 11,
 'si': 12,
 'yẹn': 13,
 'se': 14,
 'wa': 15,
 'nibe': 16,
 'kilode': 17,
 'dabọ': 18,
 'ri': 19,
 'ọ': 20,
 'titi': 21,
 'a': 22,
 'tun': 23,
 'pade': 24,
 'nigbamii': 25,
 'ilẹkun': 26,
 'de': 27,
 'silẹkun': 28}

In [15]:
sequences = tokenizer.texts_to_sequences(train_sentences)
sequences

[[11, 2],
 [8, 9, 2, 14, 15],
 [11, 2, 8, 9],
 [8, 9, 16, 3, 2],
 [17],
 [2, 18],
 [19, 20],
 [21, 22, 2, 23, 24],
 [25],
 [12, 10, 13],
 [4, 26, 13],
 [12, 10],
 [4, 10, 3, 27],
 [28],
 [5, 6, 3],
 [5, 6],
 [4, 6, 3],
 [4, 6],
 [5, 7, 3],
 [5, 7],
 [4, 7, 3],
 [4, 7]]

In [16]:
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)
padded_sequences

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0, 11,  2],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8,
         9,  2, 14, 15],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        11,  2,  8,  9],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8,
         9, 16,  3,  2],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 17],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  2, 18],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0, 19, 20],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 21,
        22,  2, 23, 24],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0, 25],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0, 12, 10, 13],
       [ 0,  0,  0,  0,  0,  0

In [17]:
model = Sequential([
  Embedding(vocab_size, embedding_dim, input_length=max_len),
  GlobalAveragePooling1D(),
  Dense(16, activation="relu"),
  Dense(16, activation="relu"),
  Dense(num_classes, activation="softmax")
])

2022-10-19 14:24:57.613995: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-19 14:24:57.614052: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-19 14:24:57.614081: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (xviix): /proc/driver/nvidia/version does not exist
2022-10-19 14:24:57.729237: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
model.compile(
  loss="sparse_categorical_crossentropy",
  optimizer='adam',
  metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 5)                 85        
                                                                 
Total params: 16,629
Trainable params: 16,629
Non-trainable params: 0
____________________________________________________

In [20]:
history = model.fit(padded_sequences, np.array(train_labels), epochs=500)

Epoch 1/500
1/1 [==============================] - 3s 3s/step - loss: 1.6095 - accuracy: 0.1818
Epoch 2/500
1/1 [==============================] - 0s 3ms/step - loss: 1.6092 - accuracy: 0.1818
Epoch 3/500
1/1 [==============================] - 0s 9ms/step - loss: 1.6089 - accuracy: 0.1818
Epoch 4/500
1/1 [==============================] - 0s 17ms/step - loss: 1.6087 - accuracy: 0.1818
Epoch 5/500
1/1 [==============================] - 0s 7ms/step - loss: 1.6084 - accuracy: 0.1818
Epoch 6/500
1/1 [==============================] - 0s 6ms/step - loss: 1.6081 - accuracy: 0.1818
Epoch 7/500
1/1 [==============================] - 0s 5ms/step - loss: 1.6078 - accuracy: 0.2727
Epoch 8/500
1/1 [==============================] - 0s 6ms/step - loss: 1.6075 - accuracy: 0.2273
Epoch 9/500
1/1 [==============================] - 0s 6ms/step - loss: 1.6072 - accuracy: 0.2273
Epoch 10/500
1/1 [==============================] - 0s 5ms/step - loss: 1.6069 - accuracy: 0.2273
Epoch 11/500
1/1 [===========

In [21]:
model.save("chat_model")

2022-10-19 14:25:21.295137: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: chat_model/assets


INFO:tensorflow:Assets written to: chat_model/assets


In [22]:
tfjs.converters.save_keras_model(model, "js model")

In [23]:
token_json = json.dumps(tokenizer.to_json())
index = tokenizer.word_index

with open("tokenizer.json", 'w') as f:
  f.write(token_json)

with open("word_dict.json", 'w') as f:
  json.dump(index, f)

# with open('label_encoder.pickle', 'wb') as enc:
#   pickle.dump(encoder, enc, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
colorama.init()

In [25]:
from colorama import Fore, Style, Back

In [26]:
def inference(input):

  # load model
  model = keras.models.load_model('chat_model')

  # load tokenizer
  with open('tokenizer.json') as file:
    tokenizer = json.load(file)
  
  tokenizer = keras.preprocessing.text.tokenizer_from_json(tokenizer)

  # load label encoder
  # with open('label_encoder.pickle', 'rb') as enc:
  #   encoder = pickle.load(enc)

  sequences = tokenizer.texts_to_sequences([input])
  padded_sequences = pad_sequences(sequences, maxlen=max_len, truncating='post')

  prediction = np.argmax(model.predict(padded_sequences))

  # pred_tag = encoder.inverse_transform([np.argmax(prediction)])
  pred_tag = intents_mapping[prediction]

  return (pred_tag)

In [27]:
inference("pa fan")

'command.fan'